<a href="https://colab.research.google.com/github/shtomar-adb/DM-Assignment-Approximate-Nearest-Neighbor/blob/main/Assignment_Approximate_Nearest_Neighbor_Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 3: Use state of art libraries for Approximate nearest neighbor search for your favorite data set

State of Art Library used fo ANN algorithms implementation: [FAISS](https://github.com/facebookresearch/faiss)

In [1]:
!apt install libomp-dev
!pip install faiss
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (216 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [2]:
!pip install lightfm


     |████████████████████████████████| 310 kB 6.6 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705353 sha256=76f93ac9df2e8a6c574270992e827fc22d6353995ad5265726112594bd99e97a
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [3]:
import pandas as pd
from lightfm import LightFM
from lightfm.datasets import fetch_movielens

__Dataset Used__: The dataset used is Movie Lens from Light FM

In [4]:
movie_lens = fetch_movielens()
df = pd.DataFrame(movie_lens)
df.head(2)

,train,test,item_features,item_feature_labels,item_labels
0,"(0, 0)\t5\n (0, 1)\t3\n (0, 2)\t4\n (0, 3...","(0, 19)\t4\n (0, 32)\t4\n (0, 60)\t4\n (0...","(0, 0)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...",Toy Story (1995),Toy Story (1995)
1,"(0, 0)\t5\n (0, 1)\t3\n (0, 2)\t4\n (0, 3...","(0, 19)\t4\n (0, 32)\t4\n (0, 60)\t4\n (0...","(0, 0)\t1.0\n (1, 1)\t1.0\n (2, 2)\t1.0\n ...",GoldenEye (1995),GoldenEye (1995)


Data Preparation for Nearest Neigbor Algorithms

In [5]:
train = movie_lens['train']
test = movie_lens['test']

model = LightFM(learning_rate=0.05, loss="warp", no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=movie_lens['item_features'], epochs=20)

item_vectors = movie_lens['item_features'] * model.item_embeddings

In [6]:
movie_names = movie_lens['item_labels']

## Section 1: LSH

In [7]:
item_vectors_lsh = item_vectors.copy()
movie_names_lsh = movie_names.copy()

In [28]:
dimention = item_vectors_lsh.shape[1]
vectors = item_vectors_lsh.astype('float32')
labels = movie_names_lsh

quantizer = faiss.IndexFlatL2(dimention)
index = faiss.IndexIVFPQ(quantizer, dimention, 8, 2, 8)
index.train(vectors)
index.add(vectors)
        

distances, indices = index.search(item_vectors_lsh[90:91], 10) 
simlar_movies_names = [labels[i] for i in indices[0]]
simlar_movies_names = '\n'.join(simlar_movies_names)
print(f"The most similar movies to {movie_names_lsh[90]} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
William Shakespeare's Romeo and Juliet (1996)
Heavy Metal (1981)
Strange Days (1995)
Abyss, The (1989)
Bram Stoker's Dracula (1992)
Monty Python's Life of Brian (1979)
Rosencrantz and Guildenstern Are Dead (1990)
Crying Game, The (1992)
Until the End of the World (Bis ans Ende der Welt) (1991)


## Section 2: Exhaustive Search

In [10]:
item_vectors_exhaustive = item_vectors.copy()
movie_names_exhaustive = movie_names.copy()

In [30]:
vectors_ex = item_vectors_exhaustive.astype('float32')
labels_ex = movie_names_exhaustive
index_ex = faiss.IndexFlatL2(item_vectors_exhaustive.shape[1])
index_ex.add(vectors_ex)

distances, indices = index_ex.search(item_vectors_exhaustive[90:91], 10) 
simlar_movies_names = [labels_ex[i] for i in indices[0]]
simlar_movies_names = '\n* '.join(simlar_movies_names)
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Heavy Metal (1981)
* William Shakespeare's Romeo and Juliet (1996)
* Fantasia (1940)
* Crow, The (1994)
* Secret Garden, The (1993)
* Winnie the Pooh and the Blustery Day (1968)
* Akira (1988)
* Ed Wood (1994)
* Bram Stoker's Dracula (1992)


## Section 4: Product Quantization

In [13]:
item_vectors_pq = item_vectors.copy()
movie_names_pq = movie_names.copy()

In [32]:
dimension_pq = item_vectors_pq.shape[1]
vectors_pq = item_vectors_pq.astype('float32')
labels_pq = movie_names_pq    
        
quantizer_pq = faiss.IndexFlatL2(dimension_pq)
index_pq = faiss.IndexIVFPQ(quantizer_pq,dimension_pq,8,2,8)
index_pq.train(vectors_pq)
index_pq.add(vectors_pq)

distances, indices = index_pq.search(item_vectors_pq[90:91], 10) 
simlar_movies_names = [labels_pq[i] for i in indices[0]]

simlar_movies_names = '\n* '.join(simlar_movies_names)
print(f"The most similar movies to {movie_names_pq[90:91]} are:\n* {simlar_movies_names}")

The most similar movies to ['Nightmare Before Christmas, The (1993)'] are:
* Nightmare Before Christmas, The (1993)
* William Shakespeare's Romeo and Juliet (1996)
* Heavy Metal (1981)
* Strange Days (1995)
* Abyss, The (1989)
* Bram Stoker's Dracula (1992)
* Monty Python's Life of Brian (1979)
* Rosencrantz and Guildenstern Are Dead (1990)
* Crying Game, The (1992)
* Until the End of the World (Bis ans Ende der Welt) (1991)


## Section 5: trees and graphs

In [16]:
!pip install annoy
import annoy

     |████████████████████████████████| 646 kB 7.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391660 sha256=0c0fdd028552717bbe416b03930ec4d6c1ceea8320f209a56591e74aa7867963
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [17]:
item_vectors_trees = item_vectors.copy()
movie_names_trees = movie_names.copy()

__Annoy__

In [36]:
dimention_trees = item_vectors_trees.shape[1]
vectors_trees = item_vectors_trees.astype('float32')
labels_trees = movie_names_trees

    
index_trees = annoy.AnnoyIndex(dimention_trees)
for i, vec in enumerate(vectors_trees):
  index_trees.add_item(i, vec.tolist())
index_trees.build(n_trees=5)

indices = index_trees.get_nns_by_vector(item_vectors_trees[90].tolist(), 10)
simlar_movies_names = [labels_trees[i] for i in indices]
simlar_movies_names = '\n* '.join(simlar_movies_names)
        
print(f"The most similar movies to {movie_names_trees[90]} are:\n* {simlar_movies_names}")  

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Heavy Metal (1981)
* Fantasia (1940)
* Beauty and the Beast (1991)
* Aladdin (1992)
* Bram Stoker's Dracula (1992)
* Interview with the Vampire (1994)
* Sound of Music, The (1965)
* Abyss, The (1989)
* Snow White and the Seven Dwarfs (1937)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


## Section 6: hnsw

In [21]:
!pip install nmslib
import nmslib

     |████████████████████████████████| 13.5 MB 76 kB/s 
     |████████████████████████████████| 188 kB 62.4 MB/s 


In [22]:
item_vectors_hnsw = item_vectors.copy()
movie_names_hnsw = movie_names.copy()

In [40]:
dimention_hnsw = item_vectors_hnsw.shape[1]
vectors_hnsw = item_vectors_hnsw.astype('float32')
labels_hnsw = movie_names_hnsw
            
index_hnsw = nmslib.init(method='hnsw', space='cosinesimil')
index_hnsw.addDataPointBatch(vectors_hnsw)
index_hnsw.createIndex({'post': 2})
        
    
indices = index_hnsw.knnQuery(item_vectors_hnsw[90], k=10)
simlar_movies_names = [labels_hnsw[i] for i in indices[0]]
simlar_movies_names = '\n* '.join(simlar_movies_names)

print(f"The most similar movies to {movie_names_hnsw[90]} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Heavy Metal (1981)
* Fantasia (1940)
* Beauty and the Beast (1991)
* Aladdin (1992)
* Jurassic Park (1993)
* Bram Stoker's Dracula (1992)
* Crow, The (1994)
* Lion King, The (1994)
* William Shakespeare's Romeo and Juliet (1996)
